# MortgageMaster for SoFi by from.rick.import.analyze Consulting 
Optimizer Code Only

Presentation Outline
Part 1 - Backend
1. Pre-Modeling EDA
2. Model Building - Hyperparameter Tuning
3. Optimizer Code Walk-thru
Part 2 - Client Pitch
4. Business Pitch and Software Prototype

## Import Packages

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl

import seaborn as sns
import dill
import random

from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_predict
from xgboost import XGBRegressor

from patsy import dmatrices, dmatrix, build_design_matrices

#adding packages for feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

# Set number of CPU cores for parallel algorithms
import os
if "CPU_LIMIT" in os.environ:
    # If you are on JupyterHub, this gives you the right number of CPUs for your virtual machine
    num_cpus = int(os.getenv("CPU_LIMIT").split('.')[0])
else:
    # If you are not on JupyterHub, this gives you the right number for your computer.
    num_cpus = os.cpu_count()

SyntaxError: invalid syntax (<ipython-input-3-c186d6bdbb91>, line 7)

In [2]:
# This sets some nicer defaults for plotting.
# This must be run in a separate cell from importing matplotlib due to a bug.
params = {'legend.fontsize': 'large',
          'figure.figsize': (11.0, 11.0),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'xx-large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'}
mpl.rcParams.update(params)

# This makes it so that the pandas dataframes don't get truncated horizontally.
pd.options.display.max_columns = 200

## Load the Data 

Import the predictions you received from your testing set. I'm calling it df_train so i don't have to edit any code that I used to engineer and impute features from our model building code.

In [3]:
df_train= pd.read_csv("PredictionsForOptimizer.csv",sep='|')

In [4]:
df_train.shape

(4930731, 38)

In [5]:
df_train.head(20)

,LOAN_IDENTIFIER,CHANNEL,SELLER_NAME,ORIGINAL_INTEREST_RATE,ORIGINAL_UPB,ORIGINAL_LOAN_TERM,ORIGINATION_DATE,FIRST_PAYMENT_DATE,ORIGINAL_LTV,ORIGINAL_COMBINED_LTV,NUMBER_OF_BORROWERS,DTI,BORROWER_CREDIT_SCORE_AT_ORIGINATION,COBORROWER_CREDIT_SCORE_AT_ORIGINATION,FIRST_TIME_HOME_BUYER_INDICATOR,LOAN_PURPOSE,PROPERTY_TYPE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,PROPERTY_STATE,MSA,ZIP_CODE_SHORT,MORTGAGE_INSURANCE_PERCENTAGE,AMORTIZATION_TYPE,MORTGAGE_INSURANCE_TYPE,RELOCATION_MORTGAGE_INDICATOR,CREDIT_SCORE_MIN,ORIGINAL_VALUE,ZERO_BALANCE_CODE,LOAN_AGE,NET_LOSS,NET_SEVERITY,LAST_STAT,LOAN_MODIFICATION_COSTS,TOTAL_LOSSES,MSA_NAME,CENSUS_2010_POP,PREDICTIONS_NET_LOSS
0,101189015,R,"Quicken Loans, LLC",3.250,94000.0,360,2020-07-20,2020-08-20,94.0,94.0,1.0,35.0,706.0,NaN,Y,P,SF,1,P,FL,0,324,25.0,FRM,1.0,N,706.0,100000.000254,NaN,8.0,0.0,0.0,00,0.0,0.0,NaN,NaN,-60.457333
1,321503282919,R,Other,3.875,201000.0,360,2016-11-20,2017-01-20,95.0,95.0,1.0,30.0,761.0,NaN,Y,P,PU,1,P,NV,29820,891,30.0,FRM,1.0,N,761.0,211578.950023,NaN,51.0,0.0,0.0,00,0.0,0.0,"Las Vegas-Henderson-Paradise, NV",1951269.0,-70.893590
2,291437756634,R,Other,5.375,39000.0,180,2003-03-20,2003-05-20,90.0,90.0,1.0,25.0,762.0,NaN,N,P,SF,1,P,MI,0,487,12.0,FRM,1.0,N,762.0,43333.334481,1.0,NaN,0.0,0.0,P,0.0,0.0,NaN,NaN,678.090724
3,980873633380,R,Other,5.250,101000.0,300,2003-07-20,2003-10-20,78.0,78.0,2.0,34.0,761.0,789.0,N,R,SF,1,P,AR,22220,727,NaN,FRM,NaN,N,761.0,129487.184237,1.0,NaN,0.0,0.0,P,0.0,0.0,"Fayetteville-Springdale-Rogers, AR-MO",463204.0,64.193035
4,435213593495,R,"Bank Of America, N.A.",5.875,160000.0,360,2003-03-20,2003-05-20,80.0,80.0,1.0,56.0,801.0,NaN,N,P,PU,1,S,NV,29820,891,NaN,FRM,NaN,N,801.0,199999.997020,1.0,NaN,0.0,0.0,P,0.0,0.0,"Las Vegas-Henderson-Paradise, NV",1951269.0,742.762276
5,911179080762,R,Other,3.875,506000.0,360,2012-06-20,2012-08-20,80.0,80.0,1.0,45.0,759.0,NaN,N,P,PU,1,P,WA,42660,980,NaN,FRM,NaN,N,759.0,632499.990575,1.0,NaN,0.0,0.0,P,0.0,0.0,"Seattle-Tacoma-Bellevue, WA",3439809.0,-56.517867
6,124443734379,R,"Bank Of America, N.A.",6.375,179000.0,360,2008-09-20,2008-10-20,80.0,95.0,2.0,30.0,747.0,798.0,N,P,PU,1,P,SC,17900,292,NaN,FRM,NaN,N,747.0,223749.996666,1.0,NaN,0.0,0.0,P,0.0,0.0,"Columbia, SC",767598.0,50.635130
7,362820423112,C,Federal Home Loan Bank Of Chicago,3.375,320000.0,180,2012-03-20,2012-05-20,77.0,80.0,1.0,32.0,813.0,NaN,N,R,SF,1,P,MA,14460,20,NaN,FRM,NaN,N,813.0,415584.425879,1.0,NaN,0.0,0.0,P,0.0,0.0,"Boston-Cambridge-Newton, MA-NH",4552402.0,-66.757753
8,105284721,R,Other,2.875,404000.0,360,2020-11-20,2021-01-20,73.0,73.0,2.0,43.0,703.0,666.0,N,R,SF,1,P,TX,19100,751,NaN,FRM,NaN,N,666.0,553424.643074,NaN,3.0,0.0,0.0,00,0.0,0.0,"Dallas-Fort Worth-Arlington, TX",6426214.0,-75.963644
9,98196561,C,"Caliber Home Loans, Inc.",3.750,385000.0,360,2019-08-20,2019-10-20,79.0,79.0,2.0,33.0,798.0,787.0,N,C,SF,1,P,CA,41740,919,NaN,FRM,NaN,N,787.0,487341.758915,1.0,NaN,0.0,0.0,P,0.0,0.0,"San Diego-Carlsbad, CA",3095313.0,-63.221321


## Summarize the Data

In [6]:
if 'ZERO_BALANCE_CODE' in df_train:
    df_train.drop(['ZERO_BALANCE_CODE', 'LOAN_AGE', 'NET_SEVERITY', 'LAST_STAT', 'LOAN_MODIFICATION_COSTS', 'TOTAL_LOSSES'],
                  axis=1,
                  inplace=True)


## Engineer Row Based Features

In [7]:
df_train['ORIGINATION_DATE'] = pd.to_datetime(df_train['ORIGINATION_DATE'], format='%Y-%m-%d')

In [8]:
#pull out quarter and month
df_train['QUARTER'] = df_train['ORIGINATION_DATE'].dt.quarter

df_train['MONTH'] = df_train['ORIGINATION_DATE'].dt.month

df_train['YEAR'] = df_train['ORIGINATION_DATE'].dt.year


In [9]:
year_train =df_train['YEAR']


In [10]:
df_train['years_since_crash'] = df_train['YEAR'] - 2008

df_train.years_since_crash

0          12
1           8
2          -5
3          -5
4          -5
           ..
4930726    -7
4930727     5
4930728    -7
4930729    -6
4930730    -4
Name: years_since_crash, Length: 4930731, dtype: int64

In [11]:
#region code
states = {
        'AK': 'O',
        'AL': 'S',
        'AR': 'S',
        'AS': 'O',
        'AZ': 'W',
        'CA': 'W',
        'CO': 'W',
        'CT': 'N',
        'DC': 'N',
        'DE': 'N',
        'FL': 'S',
        'GA': 'S',
        'GU': 'O',
        'HI': 'O',
        'IA': 'M',
        'ID': 'W',
        'IL': 'M',
        'IN': 'M',
        'KS': 'M',
        'KY': 'S',
        'LA': 'S',
        'MA': 'N',
        'MD': 'N',
        'ME': 'N',
        'MI': 'W',
        'MN': 'M',
        'MO': 'M',
        'MP': 'O',
        'MS': 'S',
        'MT': 'W',
        'NA': 'O',
        'NC': 'S',
        'ND': 'M',
        'NE': 'W',
        'NH': 'N',
        'NJ': 'N',
        'NM': 'W',
        'NV': 'W',
        'NY': 'N',
        'OH': 'M',
        'OK': 'S',
        'OR': 'W',
        'PA': 'N',
        'PR': 'O',
        'RI': 'N',
        'SC': 'S',
        'SD': 'M',
        'TN': 'S',
        'TX': 'S',
        'UT': 'W',
        'VA': 'S',
        'VI': 'O',
        'VT': 'N',
        'WA': 'W',
        'WI': 'M',
    
        'WV': 'S',
        'WY': 'W'
}


In [12]:
df_train['REGION'] = df_train['PROPERTY_STATE'].map(states)
df_train[['PROPERTY_STATE','REGION']]

,PROPERTY_STATE,REGION
0,FL,S
1,NV,W
2,MI,W
3,AR,S
4,NV,W
...,...,...
4930726,AL,S
4930727,TX,S
4930728,IL,M
4930729,MI,W


## Impute Missing Values

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin

class CategoricalImputer(BaseEstimator, TransformerMixin):
    """
    Custom defined imputer for categorical data. This allows you to specify an 
    other class where any category that doesn't meet the requirements necessary to
    be in 
    """
    
    def __init__(self, other_threshold=0, 
                 other_label="OTHER",
                 missing_first=True,
                 missing_values=np.nan, 
                 strategy='constant', 
                 fill_value="MISSING", 
                 verbose=0, 
                 copy=True, 
                 add_indicator=False):
        self.add_indicator = add_indicator
        self.copy=copy
        self.verbose=verbose
        self.fill_value=fill_value
        self.missing_first=missing_first
        self.missing_values=missing_values
        self.other_label=other_label
        self.other_threshold=other_threshold
        self.strategy=strategy
        if hasattr(missing_values, "__iter__"):
            self.missing_values = missing_values
        else:
            self.missing_values = [missing_values]
        self._imputer = SimpleImputer(missing_values=missing_values, strategy=strategy, fill_value=fill_value, verbose=verbose, copy=copy, add_indicator=False)
        self._column_categories = {}

        
    def fit(self, X, y=None):
        if type(self.other_threshold) == int or type(self.other_threshold) == float:
            other_threshold = [self.other_threshold]*len(X.columns)
        elif len(self.other_threshold) == len(X.columns):
            other_threshold = self.other_threshold
        else:
            raise TypeError("other_threshold must be either a single number or a list of numbers equal to the number of columns.")

        i = 0
        X = X.copy()
        X = X[:].astype(object)
        if self.missing_first:
            X = pd.DataFrame(self._imputer.fit_transform(X), columns=X.columns, index=X.index)
        column_categories = {}
        for column in X.columns:
            if other_threshold[i] < 1:
                other_threshold[i] = other_threshold[i]*X[column].shape[0]
            
            value_counts = X[column].value_counts()
            categories = [category for category in value_counts.index if value_counts.loc[category] >= other_threshold[i]]
            if value_counts.iloc[-1] >= other_threshold[i]:
                categories[-1] = self.other_label
            else:
                categories.append(self.other_label)
            
            self._column_categories[column] = categories
            i = i + 1
        
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        X = X[:].astype(object)
        if self.missing_first:
            X = pd.DataFrame(self._imputer.fit_transform(X), columns=X.columns, index=X.index)
        for column in X.columns:
            X.loc[~X[column].isin(self._column_categories[column]) & ~X[column].isin(self.missing_values), column] = self.other_label
        return pd.DataFrame(self._imputer.fit_transform(X), columns=X.columns, index=X.index)[:].astype(str)

In [14]:
continuous_mean = ['ORIGINAL_INTEREST_RATE',
                   'ORIGINAL_UPB',
                   'ORIGINAL_LOAN_TERM',
                   'ORIGINAL_LTV',
                   'ORIGINAL_COMBINED_LTV',
                   'DTI',
                   'ORIGINAL_VALUE',
                   'YEAR','years_since_crash']

continuous_zero = ['MORTGAGE_INSURANCE_PERCENTAGE', 
#                    'BORROWER_CREDIT_SCORE_AT_ORIGINATION',
#                    'COBORROWER_CREDIT_SCORE_AT_ORIGINATION',
                   'CREDIT_SCORE_MIN']

special = ['BORROWER_CREDIT_SCORE_AT_ORIGINATION','COBORROWER_CREDIT_SCORE_AT_ORIGINATION']


continuous_variables = continuous_mean + continuous_zero + special

categorical_variables = ['CHANNEL',
                         'SELLER_NAME',
                         'NUMBER_OF_BORROWERS',
                         'FIRST_TIME_HOME_BUYER_INDICATOR',
                         'LOAN_PURPOSE',
                         'PROPERTY_TYPE',
                         'NUMBER_OF_UNITS',
                         'OCCUPANCY_STATUS',
                         'PROPERTY_STATE',
                         'ZIP_CODE_SHORT',
                         'AMORTIZATION_TYPE',
                         'MORTGAGE_INSURANCE_TYPE',
                         'RELOCATION_MORTGAGE_INDICATOR',
                         'MSA',
                         'MSA_NAME',
                         'CENSUS_2010_POP', 'REGION']

## Feature Engineering for Tree Based Models

In [15]:
continuous_features_trees = ['ORIGINAL_INTEREST_RATE',
                             'ORIGINAL_UPB',
                             'ORIGINAL_LOAN_TERM',
                             'ORIGINAL_LTV',
                             'ORIGINAL_COMBINED_LTV',
                             'DTI',
                             'BORROWER_CREDIT_SCORE_AT_ORIGINATION',
                             'COBORROWER_CREDIT_SCORE_AT_ORIGINATION',
                             'MORTGAGE_INSURANCE_PERCENTAGE',
                             'CREDIT_SCORE_MIN',
                             'ORIGINAL_VALUE','years_since_crash'
                            ]

In [16]:
cat_ordinal_features_trees = ['SELLER_NAME', 'NUMBER_OF_BORROWERS', 'PROPERTY_STATE','CENSUS_2010_POP','NUMBER_OF_UNITS']
cat_dummy_features_trees = ['FIRST_TIME_HOME_BUYER_INDICATOR','CHANNEL','LOAN_PURPOSE','PROPERTY_TYPE','OCCUPANCY_STATUS','AMORTIZATION_TYPE','MORTGAGE_INSURANCE_TYPE', 'RELOCATION_MORTGAGE_INDICATOR','QUARTER', 'MONTH', 'YEAR', 'REGION']

## Data Prep

In [17]:
imputer_mean_final = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer_zero_final = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
categorical_imputer_final = CategoricalImputer(other_threshold=.01)

In [18]:
imputer_mean_final.fit(df_train[continuous_mean])
df_train[continuous_mean] = imputer_mean_final.transform(df_train[continuous_mean])

In [19]:
imputer_zero_final.fit(df_train[continuous_zero])
df_train[continuous_zero] = imputer_zero_final.transform(df_train[continuous_zero])


In [20]:
categorical_imputer_final.fit(df_train[categorical_variables])
df_train[categorical_variables] = categorical_imputer_final.transform(df_train[categorical_variables])


In [21]:
imputer_special_final = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=700)
imputer_special_final.fit(df_train[special])
df_train[special] = imputer_special_final.transform(df_train[special])

In [22]:
def summarize_dataframe(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.DataFrame({'Variable Name': df.columns,
                                   'Data Type': df.dtypes,
                                   'Missing Values': df.isnull().sum(),
                                   'Unique Values': [df[name].nunique() for name in df.columns]}
                                 ).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all', datetime_is_numeric=True).transpose()], axis=1).fillna(""))

## Set a Threshold and Compute Revenue - Proof of Concept for Optimized thresholds for different segments

Here we are testing two ideas: 
1. proof of concept that a solver could be used to set an optimal threshold and improve revenue
2. proof of concept that dividing loans into tranches based on SOME feature will improve revenue even further

### Below is an optimization for a threshold over the whole data set we loaded

In [21]:
# Coding the objective function
def f(threshold):
    def accept_by_predicted_loss(predictions, threshold):
        return predictions <= threshold
    def loan_loss_revenue(loans_accepted):
        revenue = (df_train[loan_decisions]['ORIGINAL_VALUE']*.005).sum() - df_train[loan_decisions]['NET_LOSS'].sum()
        return revenue/1000000
    loan_decisions = accept_by_predicted_loss(df_train['PREDICTIONS_NET_LOSS'], threshold)
    x=loan_loss_revenue(loan_decisions)
    return -x
    #-x since this is a minimizer function, could not find a maximizer

In [35]:
#importing minimize_scalar for univariate optimization
from scipy.optimize import minimize_scalar

In [23]:
res = minimize_scalar(f)
res

     fun: -6888.873494218317
    nfev: 36
     nit: 26
 success: True
       x: 1317.9527431376123

In [24]:
#revenue if a single threshold is applied to all loans (remember we need to undo the sign change to show positive)
single_threshold_rev = -1*res.fun
single_threshold_rev

6888.873494218317

# Helping SoFi add an incremental segment of consumer

## Business Facts/Assumptions
\
Two main business opportunities:

1. Identify highest revenue generating loan segments (ie by prorperty type, loan purpose, region, etc).
2. Identify loans that SoFi could hold onto vs sell immediately to the secondary market based on profitability

### Facts
- SoFi has a tendency to service and lend to users on the higher end of the spectrum.
- Reported average credit score for user is 720
- Let's assume for this example that they currently only tap into and lend to the 690+ credit score market
- The dataset we are using here shows that about 84% of mortgages have borrowers with credit scores of 690+ (see back of envelope math below)

### Assumptions (So SoFi Q3 assumptions)
- In 2020 SoFi grew lending product customers by 120K.  Revenue wise, Home Mortgage products revenue was the only segment that grew YoY. Let's assume that 100K of the new lending product consumers were home loan originators.
- Let's assume SoFi turns down loan applicants with scores less than 690 (around 84% of applications.  This means that in total, SoFi received around 120K applications.

Let's simulate this new group of applications to put a dollar value on the opportunity we can provide with our model predictions


## Back of the envelope math - understand what percent of loans were originated by folks with 690+ credit scores

In [24]:
strong_credit = df_train[df_train['BORROWER_CREDIT_SCORE_AT_ORIGINATION']>=690]

In [25]:
above690=strong_credit.shape[0]

In [26]:
total_loans=df_train.shape[0]

In [27]:
pct_above_690 = above690/total_loans
pct_above_690

0.8411927156439887

## Let's simulate 1 year's worth of loan applications by sampling 120k records as "loan applications" from our dataset and then pair down applications by credit score

In [28]:
simulated_apps = df_train.sample(120000, random_state=201)
simulated_apps.shape

(120000, 37)

Now let's simulate only accepting mortgage applications with credit scores about 690

In [29]:
target_segment = simulated_apps[simulated_apps['BORROWER_CREDIT_SCORE_AT_ORIGINATION']>=690]
target_segment.shape

(100982, 37)

In [30]:
np.sum(df_train['YEAR']>=2019)

818417

In [31]:
sofi_presentation = df_train.sample(200000, random_state=201)
sofi_presentation.to_csv('sofi_biz_case_df.csv', sep='|')

# Picking an optimal threshold to improve revenue for SoFi's current target segment

### Baseline - Revenue accepting all apps from consumers in target_segment. This is revenue number we want to beat using our predictions
#### $131M

In [32]:
all_target_segment_rev = ((target_segment['ORIGINAL_VALUE']*.005).sum() - target_segment['NET_LOSS'].sum())/1000000
all_target_segment_rev

131.05725215085425

In [33]:
def f_target_segment(threshold):
    def accept_by_predicted_loss(predictions, threshold):
        return predictions <= threshold
    def loan_loss_revenue(loans_accepted):
        revenue = (target_segment[loan_decisions]['ORIGINAL_VALUE']*.005).sum() - target_segment[loan_decisions]['NET_LOSS'].sum()
        return revenue/1000000
    loan_decisions = accept_by_predicted_loss(target_segment['PREDICTIONS_NET_LOSS'], threshold)
    x=loan_loss_revenue(loan_decisions)
    return -x
    #-x since this is a minimizer function, could not find a maximizer

In [36]:
target_segment_optimizer = minimize_scalar(f_target_segment)
target_segment_optimizer

     fun: -151.81105040048
    nfev: 38
     nit: 20
 success: True
       x: 1224.3731297674158

## Recommendation 1: If SoFi continues to focus on high credit score borrowers, set a predicted net_loss threshold of 1224 and increase revenue by $21M 

In [37]:
optimized_ts_rev=target_segment_optimizer.fun*-1
optimized_ts_rev

151.81105040048

In [38]:
revenue_increase = optimized_ts_rev-all_target_segment_rev
revenue_increase

20.75379824962576

This looks good at face value, but the trade off is that further limiting an already narrower target segment (690+ credit scores) means less TOTAL sofi customers to cross-sell other SoFi services and products to. This idea, is Low Risk, Low reward. 

# Incremental target consumer opportunity - take on a slightly riskier customer portfolio, but hedge using tranches based on DTI

What if SoFi were to expand its consumer base to slightly riskier borrowers? First, let's calculate a baseline revenue if SoFi were to accept all 120K simulated loan applications (in other words, not setting a credit score floor for their target customers)

In [39]:
all_apps_rev = ((simulated_apps['ORIGINAL_VALUE']*.005).sum() - simulated_apps['NET_LOSS'].sum())/1000000
all_apps_rev

124.29814203688153

-With no thresholds, accepting all apps would indeed be a bad idea.  Rev would go from 131M from just the target segment, down to 124M if we accepted all applications. Good to know. \
-Rather than try to convince our client that they should suddenly take on the entire spectrum of riskier buyers, let's take baby steps and extend the target consumer down to borrowers with 650+ credit scores \
-Our hypothesis on 650+ is that there are a mix of good borrowers with credit scores that have not matured yet but also some bad borrowers in the 650-690 segments.  \
If we can optimize a threshold that picks the best of these borrowers, then we should gain incremental revenu from an incremental user segment that we can cross-sell into other so-fi products.

Let's baseline revenue of this new target segment (called s2)

In [40]:
s2 = simulated_apps[simulated_apps['BORROWER_CREDIT_SCORE_AT_ORIGINATION']>=650]
s2.shape

(113293, 37)

In [41]:
s2_rev = ((s2['ORIGINAL_VALUE']*.005).sum() - s2['NET_LOSS'].sum())/1000000
s2_rev

130.38001854987513

Slight decline in revenue (-1M), but that's to be expected if we accpet all apps in this slightly riskier portfolio. Let's optimize by building tranches for credit scores in our new expanded segment

# Optimize Thresholds by segmenting loans into groups

In [42]:
# bins for DTI groups were made by using tableau's cluster analysis function
bins = [-np.inf,15,23,30,38,45,53,np.inf]
labels = [1,2,3,4,5,6,7]


In [43]:
s2['dti_tranch'] = pd.cut(s2['DTI'],bins=bins,labels=labels)

/tmp/ipykernel_145/1614762156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s2['dti_tranch'] = pd.cut(s2['DTI'],bins=bins,labels=labels)


In [44]:
#instantiating empty lists to load in results of optimizer
revenue=[]
limits=[]


In [45]:
bin1=s2[s2['dti_tranch']==1]
bin2=s2[s2['dti_tranch']==2]
bin3=s2[s2['dti_tranch']==3]
bin4=s2[s2['dti_tranch']==4]
bin5=s2[s2['dti_tranch']==5]
bin6=s2[s2['dti_tranch']==6]
bin7=s2[s2['dti_tranch']==7]

In [46]:
def f(threshold):
    def accept_by_predicted_loss(predictions, threshold):
        return predictions <= threshold
    def loan_loss_revenue(loans_accepted):
        revenue = (bin1[loan_decisions]['ORIGINAL_VALUE']*.005).sum() - bin1[loan_decisions]['NET_LOSS'].sum()
        return revenue/1000000
    loan_decisions = accept_by_predicted_loss(bin1['PREDICTIONS_NET_LOSS'], threshold)
    x=loan_loss_revenue(loan_decisions)
    return -x
    #-x since this is a minimizer function, could not find a maximizer

In [47]:
func1=minimize_scalar(f)
revenue.append(-1*func1.fun)
limits.append(func1.x)
bin1_thresh=func1.x

### Above code is repeated for all seven DTI groups

In [48]:
def f(threshold):
    def accept_by_predicted_loss(predictions, threshold):
        return predictions <= threshold
    def loan_loss_revenue(loans_accepted):
        revenue = (bin2[loan_decisions]['ORIGINAL_VALUE']*.005).sum() - bin2[loan_decisions]['NET_LOSS'].sum()
        return revenue/1000000
    loan_decisions = accept_by_predicted_loss(bin2['PREDICTIONS_NET_LOSS'], threshold)
    x=loan_loss_revenue(loan_decisions)
    return -x
    #-x since this is a minimizer function, could not find a maximizer

In [49]:
func2=minimize_scalar(f)
revenue.append(-1*func2.fun)
limits.append(func2.x)
bin2_thresh=func2.x

In [50]:
def f(threshold):
    def accept_by_predicted_loss(predictions, threshold):
        return predictions <= threshold
    def loan_loss_revenue(loans_accepted):
        revenue = (bin3[loan_decisions]['ORIGINAL_VALUE']*.005).sum() - bin3[loan_decisions]['NET_LOSS'].sum()
        return revenue/1000000
    loan_decisions = accept_by_predicted_loss(bin3['PREDICTIONS_NET_LOSS'], threshold)
    x=loan_loss_revenue(loan_decisions)
    return -x
    #-x since this is a minimizer function, could not find a maximizer

In [51]:
func3=minimize_scalar(f)
revenue.append(-1*func3.fun)
limits.append(func3.x)
bin3_thresh=func3.x

In [52]:
def f(threshold):
    def accept_by_predicted_loss(predictions, threshold):
        return predictions <= threshold
    def loan_loss_revenue(loans_accepted):
        revenue = (bin4[loan_decisions]['ORIGINAL_VALUE']*.005).sum() - bin4[loan_decisions]['NET_LOSS'].sum()
        return revenue/1000000
    loan_decisions = accept_by_predicted_loss(bin4['PREDICTIONS_NET_LOSS'], threshold)
    x=loan_loss_revenue(loan_decisions)
    return -x
    #-x since this is a minimizer function, could not find a maximizer

In [53]:
func4=minimize_scalar(f)
revenue.append(-1*func4.fun)
limits.append(func4.x)
bin4_thresh=func4.x

In [54]:
def f(threshold):
    def accept_by_predicted_loss(predictions, threshold):
        return predictions <= threshold
    def loan_loss_revenue(loans_accepted):
        revenue = (bin5[loan_decisions]['ORIGINAL_VALUE']*.005).sum() - bin5[loan_decisions]['NET_LOSS'].sum()
        return revenue/1000000
    loan_decisions = accept_by_predicted_loss(bin5['PREDICTIONS_NET_LOSS'], threshold)
    x=loan_loss_revenue(loan_decisions)
    return -x
    #-x since this is a minimizer function, could not find a maximizer

In [55]:
func5=minimize_scalar(f)
revenue.append(-1*func5.fun)
limits.append(func5.x)
bin5_thresh=func5.x

In [56]:
def f(threshold):
    def accept_by_predicted_loss(predictions, threshold):
        return predictions <= threshold
    def loan_loss_revenue(loans_accepted):
        revenue = (bin6[loan_decisions]['ORIGINAL_VALUE']*.005).sum() - bin6[loan_decisions]['NET_LOSS'].sum()
        return revenue/1000000
    loan_decisions = accept_by_predicted_loss(bin6['PREDICTIONS_NET_LOSS'], threshold)
    x=loan_loss_revenue(loan_decisions)
    return -x
    #-x since this is a minimizer function, could not find a maximizer

In [57]:
func6=minimize_scalar(f)
revenue.append(-1*func6.fun)
limits.append(func6.x)
bin6_thresh=func6.x

In [58]:
def f(threshold):
    def accept_by_predicted_loss(predictions, threshold):
        return predictions <= threshold
    def loan_loss_revenue(loans_accepted):
        revenue = (bin7[loan_decisions]['ORIGINAL_VALUE']*.005).sum() - bin7[loan_decisions]['NET_LOSS'].sum()
        return revenue/1000000
    loan_decisions = accept_by_predicted_loss(bin7['PREDICTIONS_NET_LOSS'], threshold)
    x=loan_loss_revenue(loan_decisions)
    return -x
    #-x since this is a minimizer function, could not find a maximizer

In [59]:
func7=minimize_scalar(f)
revenue.append(-1*func7.fun)
limits.append(func7.x)
bin7_thresh=func7.x

In [60]:
# this code makes it easier for me to segment loans that were accepted and not accepted based on optimized thresholds
bin1['is_accept']= bin1['PREDICTIONS_NET_LOSS']<=bin1_thresh
bin2['is_accept']= bin2['PREDICTIONS_NET_LOSS']<=bin2_thresh
bin3['is_accept']= bin3['PREDICTIONS_NET_LOSS']<=bin3_thresh
bin4['is_accept']= bin4['PREDICTIONS_NET_LOSS']<=bin4_thresh
bin5['is_accept']= bin5['PREDICTIONS_NET_LOSS']<=bin5_thresh
bin6['is_accept']= bin6['PREDICTIONS_NET_LOSS']<=bin6_thresh
bin7['is_accept']= bin7['PREDICTIONS_NET_LOSS']<=bin6_thresh

/tmp/ipykernel_145/2416543040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bin1['is_accept']= bin1['PREDICTIONS_NET_LOSS']<=bin1_thresh
/tmp/ipykernel_145/2416543040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bin2['is_accept']= bin2['PREDICTIONS_NET_LOSS']<=bin2_thresh
/tmp/ipykernel_145/2416543040.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [61]:
print(revenue)
print(limits)

[9.343883125997703, 25.187994903410786, 32.69164422291956, 41.45042334447426, 35.803415500179845, 14.616667017401555, 3.0905502389047284]
[295.06506221553644, 1126.124163087947, 1567.4391318361963, 1159.710690622944, 319.53735741165826, 1825.9022936666138, 848.7360371933062]


In [62]:
dti_tranched_revenue=np.sum(revenue)

In [63]:
print(dti_tranched_revenue)
print(optimized_ts_rev)

162.18457835328846
151.81105040048


## Recommendation 2: 
- Expand mortgage product segment to include mortgage applicants with credit score and optimize thresholds by tranching loan applications by DTI ratio of applicants. 
- This recommendation will increase revenues by 31.1M vs SoFi's current target segment and 10.3M over an optimized portfolio of their current segment.

In [64]:
dti_tranched_revenue-all_target_segment_rev

31.127326202434205

In [65]:
dti_tranched_revenue-optimized_ts_rev


10.373527952808445

# Export dataframe with thresholds

In [115]:
bin_dfs = [bin1,bin2,bin3,bin4,bin5,bin6,bin7]
sofi_biz_case_df = pd.concat(bin_dfs)
sofi_biz_case_df

,LOAN_IDENTIFIER,CHANNEL,SELLER_NAME,ORIGINAL_INTEREST_RATE,ORIGINAL_UPB,ORIGINAL_LOAN_TERM,ORIGINATION_DATE,FIRST_PAYMENT_DATE,ORIGINAL_LTV,ORIGINAL_COMBINED_LTV,NUMBER_OF_BORROWERS,DTI,BORROWER_CREDIT_SCORE_AT_ORIGINATION,COBORROWER_CREDIT_SCORE_AT_ORIGINATION,FIRST_TIME_HOME_BUYER_INDICATOR,LOAN_PURPOSE,PROPERTY_TYPE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,PROPERTY_STATE,MSA,ZIP_CODE_SHORT,MORTGAGE_INSURANCE_PERCENTAGE,AMORTIZATION_TYPE,MORTGAGE_INSURANCE_TYPE,RELOCATION_MORTGAGE_INDICATOR,CREDIT_SCORE_MIN,ORIGINAL_VALUE,NET_LOSS,MSA_NAME,CENSUS_2010_POP,PREDICTIONS_NET_LOSS,QUARTER,MONTH,YEAR,years_since_crash,REGION,ltv_tranch,dti_tranch,is_accept
385021,148134585489,R,"Citimortgage, Inc.",7.000,208000.0,180.0,2002-02-20,2002-04-20,80.0,80.0,2.0,10.0,769.0,700.0,N,C,SF,1,P,OTHER,OTHER,OTHER,0.0,OTHER,MISSING,N,769.0,259999.996126,0.0,OTHER,OTHER,-73.977206,1,2,2002.0,-6.0,M,4,1,True
3218322,725786405164,C,OTHER,3.250,218000.0,240.0,2016-09-20,2016-11-20,60.0,60.0,2.0,13.0,809.0,801.0,N,R,SF,1,P,WA,42660,OTHER,0.0,OTHER,MISSING,N,801.0,363333.318896,0.0,"Seattle-Tacoma-Bellevue, WA",3439809.0,-73.736914,3,9,2016.0,8.0,W,2,1,True
1123388,254301736234,R,Other,4.125,267000.0,360.0,2018-01-20,2018-03-20,80.0,80.0,2.0,11.0,772.0,800.0,Y,P,PU,1,P,OTHER,0,OTHER,0.0,OTHER,MISSING,N,772.0,333749.995027,0.0,MISSING,MISSING,-69.567862,1,1,2018.0,10.0,W,4,1,True
55209,334920256925,OTHER,"Citimortgage, Inc.",5.625,122000.0,240.0,2005-04-20,2005-05-20,73.0,73.0,1.0,15.0,702.0,700.0,Y,P,CO,1,P,MA,14460,OTHER,0.0,OTHER,MISSING,N,702.0,167123.283305,0.0,"Boston-Cambridge-Newton, MA-NH",4552402.0,2555.299889,2,4,2005.0,-3.0,N,3,1,False
741425,613076297217,R,OTHER,5.875,164000.0,180.0,2007-04-20,2007-06-20,64.0,64.0,2.0,15.0,764.0,790.0,N,C,SF,1,P,TN,OTHER,OTHER,0.0,OTHER,MISSING,N,764.0,256250.005728,0.0,OTHER,OTHER,-210.491405,2,4,2007.0,-1.0,S,3,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4306267,532778037247,R,Other,5.500,323000.0,180.0,2003-03-20,2003-05-20,40.0,50.0,2.0,55.0,709.0,698.0,N,C,SF,1,P,NY,35620,OTHER,0.0,OTHER,MISSING,N,698.0,807499.987967,0.0,"New York-Newark-Jersey City, NY-NJ-PA",19567410.0,-109.192247,1,3,2003.0,-5.0,N,1,7,True
1120618,345942115511,C,"Bank Of America, N.A.",6.750,234000.0,360.0,2006-06-20,2006-08-20,36.0,36.0,2.0,59.0,813.0,785.0,N,P,PU,1,P,MD,47900,OTHER,0.0,OTHER,MISSING,N,785.0,649999.974171,0.0,"Washington-Arlington-Alexandria, DC-VA-MD-WV",5636232.0,-297.933844,2,6,2006.0,-2.0,N,1,7,True
4456765,749993080076,C,OTHER,7.250,125000.0,360.0,2000-12-20,2001-02-20,95.0,95.0,1.0,62.0,685.0,700.0,N,P,SF,1,P,GA,0,OTHER,25.0,OTHER,1.0,N,685.0,131578.949020,0.0,MISSING,MISSING,70.883699,4,12,2000.0,-8.0,S,4,7,True
4114558,999670278730,R,"Wells Fargo Bank, N.A.",6.150,251000.0,360.0,2005-10-20,2005-12-20,58.0,58.0,1.0,64.0,676.0,700.0,N,R,SF,1,P,CA,OTHER,945,0.0,OTHER,MISSING,N,676.0,432758.633142,0.0,OTHER,OTHER,5751.696951,4,10,2005.0,-3.0,W,2,7,False


In [116]:
sofi_biz_case_df.to_csv('sofi_biz_case_df.csv', sep='|')